In [21]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

True
NVIDIA GeForce RTX 3060 Laptop GPU
1


In [73]:
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.utils
from PIL import Image
import glob
import sys, os
sys.path.append(os.pardir) # 부모 디렉토리의 파일을 가져올 수 있도록 설정
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from collections import OrderedDict
import pickle
import torch
%matplotlib inline
import cv2
from imutils.video import VideoStream
from pathlib import Path
import torch
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
import torch.nn as nn
from PIL import Image
from torch.autograd import Variable
import PIL.ImageOps
from torch import optim
import torch.nn.functional as F
import random
import numpy as np

In [23]:
print(f"현재 디렉토리 위치: {os.getcwd()}")

현재 디렉토리 위치: C:\Users\yuddo\Yuchul\Project


In [24]:
#folder = "Project"
#project_dir = "Bowling"
#base_path = Path("C:\\Users\yuddo\Yuchul")
#project_path = base_path / folder / project_dir
#os.chdir(project_path)
#for x in list(project_path.glob("*")):
#    if x.is_dir():
#        dir_name = str(x.relative_to(project_path))
#        os.rename(dir_name, dir_name.split(" ", 1)[0])
#print(f"현재 디렉토리 위치: {os.getcwd()}")

In [25]:
current_path = Path().absolute()
data_path = current_path / "data"

In [26]:
print("현재 디렉토리 위치: {}".format(current_path))

현재 디렉토리 위치: C:\Users\yuddo\Yuchul\Project


In [27]:
if (data_path / "user").exists():
    print("폴더가 있습니다! 이어서 진행하세요~")
else: print("없습니다")

없습니다


In [28]:
user_dir = './data/user'

In [29]:
output_dir = './data/output'

In [30]:
def output_keypoints(frame, net, threshold, BODY_PARTS, now_frame, total_frame):
    global points
    
    # 입력 이미지 사이즈 정의
    image_height = 368
    image_width = 368
    
    # 네트워크에 넣기 위한 전처리
    input_blob = cv2.dnn.blobFromImage(frame, 1.0/255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)
    
    # 전처리된 blob 네트워크에 입력
    net.setInput(input_blob)

    # 결과 받아오기
    out = net.forward()
    # The output is a 4D matrix :
    # The first dimension being the image ID ( in case you pass more than one image to the network ).
    # The second dimension indicates the index of a keypoint.
    # The model produces Confidence Maps and Part Affinity maps which are all concatenated.
    # For COCO model it consists of 57 parts – 18 keypoint confidence Maps + 1 background + 19*2 Part Affinity Maps. Similarly, for MPI, it produces 44 points.
    # We will be using only the first few points which correspond to Keypoints.
    # The third dimension is the height of the output map.
    out_height = out.shape[2]
    # The fourth dimension is the width of the output map.
    out_width = out.shape[3]

    # 원본 이미지의 높이, 너비를 받아오기
    frame_height, frame_width = frame.shape[:2]

    # 포인트 리스트 초기화
    points = []

    print(f"============================== frame: {now_frame:.0f} / {total_frame:.0f} ==============================")
    for i in range(len(BODY_PARTS)):

        # 신체 부위의 confidence map
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        # 원본 이미지에 맞게 포인트 위치 조정
        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)

        if prob > threshold:  # [pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

            points.append((x, y))
            print(f"[pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

        else:  # [not pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

            points.append(None)
            print(f"[not pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

    return frame

In [31]:
def output_keypoints_with_lines(frame, POSE_PAIRS):
    for pair in POSE_PAIRS:
        part_a = pair[0]  # 0 (Head)
        part_b = pair[1]  # 1 (Neck)
        if points[part_a] and points[part_b]:
            print(f"[linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")
            cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 3)
        else:
            print(f"[not linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")

    return frame

In [45]:
def output_keypoints_with_lines_video(proto_file, weights_file, video_path, threshold, BODY_PARTS, POSE_PAIRS):

    # 비디오 읽어오기
    capture = cv2.VideoCapture(video_path)
    
    # 네트워크 불러오기
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
    
    # GPU 사용
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    if capture.isOpened():
        fps = capture.get(cv2.CAP_PROP_FPS)
        frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        output = cv2.VideoWriter("output.avi", fourcc, fps, (frame_width, frame_height))
    
    while True:
        now_frame_boy = capture.get(cv2.CAP_PROP_POS_FRAMES)
        total_frame_boy = capture.get(cv2.CAP_PROP_FRAME_COUNT)

        if now_frame_boy == total_frame_boy:
            break

        ret, frame_boy = capture.read()
        frame_boy = output_keypoints(frame=frame_boy, net=net, threshold=threshold, BODY_PARTS=BODY_PARTS, now_frame=now_frame_boy, total_frame=total_frame_boy)
        frame_boy = output_keypoints_with_lines(frame=frame_boy, POSE_PAIRS=POSE_PAIRS)
        cv2.imshow("Output_Keypoints", frame_boy)
        output.write(frame_boy)
        
        if cv2.waitKey(10) == 27:  # esc 입력시 종료
            break

    capture.release()
    output.release()
    print("Output Video saved successfully")
    cv2.destroyAllWindows()


In [48]:
BODY_PARTS_MPI = {0: "Head", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                  5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                  10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "Chest",
                  15: "Background"}

POSE_PAIRS_MPI = [[0, 1], [1, 2], [1, 5], [1, 14], [2, 3], [3, 4], [5, 6],
                  [6, 7], [8, 9], [9, 10], [11, 12], [12, 13], [14, 8], [14, 11]]

BODY_PARTS_COCO = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                   5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                   10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "REye",
                   15: "LEye", 16: "REar", 17: "LEar", 18: "Background"}

POSE_PAIRS_COCO = [[0, 1], [0, 14], [0, 15], [1, 2], [1, 5], [1, 8], [1, 11], [2, 3], [3, 4],
                   [5, 6], [6, 7], [8, 9], [9, 10], [12, 13], [11, 12], [14, 16], [15, 17]]

BODY_PARTS_BODY_25 = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                      5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "MidHip", 9: "RHip",
                      10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee", 14: "LAnkle",
                      15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe",
                      20: "LSmallToe", 21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"}

POSE_PAIRS_BODY_25 = [[0, 1], [0, 15], [0, 16], [1, 2], [1, 5], [1, 8], [8, 9], [8, 12], [9, 10], [12, 13], [2, 3],
                      [3, 4], [5, 6], [6, 7], [10, 11], [13, 14], [15, 17], [16, 18], [14, 21], [19, 21], [20, 21],
                      [11, 24], [22, 24], [23, 24]]

# 신경 네트워크의 구조를 지정하는 prototxt 파일 (다양한 계층이 배열되는 방법 등)
protoFile_mpi = "C:/Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\mpi\pose_deploy_linevec.prototxt"
protoFile_mpi_faster = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\mpi\pose_deploy_linevec_faster_4_stages.prototxt"
protoFile_coco = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\coco\pose_deploy_linevec.prototxt"
protoFile_body_25 = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose-master\models\pose\body_25\pose_deploy.prototxt"

# 훈련된 모델의 weight 를 저장하는 caffemodel 파일
weightsFile_mpi = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\mpi\pose_iter_160000.caffemodel"
weightsFile_coco = "C:\\Users\yuddo\Yuchul\Project\Bowling\openpose\models\pose\coco\pose_iter_440000.caffemodel"
weightsFile_body_25 = "C:/Users/yuddo/Yuchul/Project/Bowling/openpose-master/models/pose/body_25/pose_iter_584000.caffemodel"

# 비디오 경로
man = "C:/Users/yuddo/Yuchul/Project/Bowling/data/pro/pro_1.mp4"

# 키포인트를 저장할 빈 리스트
points = []

#output_keypoints_with_lines_video(proto_file=protoFile_mpi_faster, weights_file=weightsFile_mpi, video_path=man,
#                                 threshold=0.1, BODY_PARTS=BODY_PARTS_MPI, POSE_PAIRS=POSE_PAIRS_MPI)

output_keypoints_with_lines_video(proto_file=protoFile_coco, weights_file=weightsFile_coco, video_path=man,
                                  threshold=0.1, BODY_PARTS=BODY_PARTS_COCO, POSE_PAIRS=POSE_PAIRS_COCO)

#output_keypoints_with_lines_video(proto_file=protoFile_body_25, weights_file=weightsFile_body_25, video_path=man,
#                                 threshold=0.1, BODY_PARTS=BODY_PARTS_BODY_25, POSE_PAIRS=POSE_PAIRS_BODY_25)

============================== frame: 0 / 153 ==============================
[pointed] Nose (0) => prob: 0.55543 / x: 359 / y: 140
[pointed] Neck (1) => prob: 0.66557 / x: 252 / y: 187
[pointed] RShoulder (2) => prob: 0.52145 / x: 332 / y: 211
[pointed] RElbow (3) => prob: 0.63476 / x: 372 / y: 352
[not pointed] RWrist (4) => prob: 0.08111 / x: 385 / y: 234
[pointed] LShoulder (5) => prob: 0.57784 / x: 172 / y: 187
[pointed] LElbow (6) => prob: 0.53797 / x: 119 / y: 328
[not pointed] LWrist (7) => prob: 0.02785 / x: 159 / y: 399
[pointed] RHip (8) => prob: 0.32294 / x: 266 / y: 469
[pointed] RKnee (9) => prob: 0.47944 / x: 292 / y: 610
[pointed] RAnkle (10) => prob: 0.49958 / x: 252 / y: 821
[pointed] LHip (11) => prob: 0.34894 / x: 172 / y: 446
[pointed] LKnee (12) => prob: 0.50136 / x: 226 / y: 610
[pointed] LAnkle (13) => prob: 0.34204 / x: 266 / y: 774
[pointed] REye (14) => prob: 0.67848 / x: 359 / y: 117
[not pointed] LEye (15) => prob: 0.03699 / x: 359 / y: 117
[pointed] REar (1

============================== frame: 5 / 153 ==============================
[pointed] Nose (0) => prob: 0.51510 / x: 399 / y: 140
[pointed] Neck (1) => prob: 0.61257 / x: 292 / y: 211
[pointed] RShoulder (2) => prob: 0.43933 / x: 359 / y: 211
[pointed] RElbow (3) => prob: 0.57879 / x: 425 / y: 352
[not pointed] RWrist (4) => prob: 0.05934 / x: 359 / y: 375
[pointed] LShoulder (5) => prob: 0.66737 / x: 212 / y: 187
[pointed] LElbow (6) => prob: 0.62498 / x: 159 / y: 328
[not pointed] LWrist (7) => prob: 0.05392 / x: 199 / y: 399
[pointed] RHip (8) => prob: 0.36418 / x: 306 / y: 493
[pointed] RKnee (9) => prob: 0.39187 / x: 319 / y: 633
[pointed] RAnkle (10) => prob: 0.59182 / x: 306 / y: 845
[pointed] LHip (11) => prob: 0.32447 / x: 199 / y: 469
[pointed] LKnee (12) => prob: 0.57662 / x: 252 / y: 610
[pointed] LAnkle (13) => prob: 0.53794 / x: 279 / y: 798
[pointed] REye (14) => prob: 0.58915 / x: 399 / y: 117
[not pointed] LEye (15) => prob: 0.02868 / x: 399 / y: 117
[pointed] REar (1

============================== frame: 10 / 153 ==============================
[pointed] Nose (0) => prob: 0.21432 / x: 399 / y: 117
[pointed] Neck (1) => prob: 0.63403 / x: 292 / y: 211
[pointed] RShoulder (2) => prob: 0.51555 / x: 372 / y: 234
[pointed] RElbow (3) => prob: 0.68212 / x: 425 / y: 352
[pointed] RWrist (4) => prob: 0.15160 / x: 359 / y: 375
[pointed] LShoulder (5) => prob: 0.66563 / x: 212 / y: 187
[pointed] LElbow (6) => prob: 0.58861 / x: 172 / y: 328
[not pointed] LWrist (7) => prob: 0.04552 / x: 199 / y: 399
[pointed] RHip (8) => prob: 0.34934 / x: 319 / y: 493
[pointed] RKnee (9) => prob: 0.26844 / x: 319 / y: 610
[pointed] RAnkle (10) => prob: 0.62674 / x: 319 / y: 821
[pointed] LHip (11) => prob: 0.32605 / x: 212 / y: 469
[pointed] LKnee (12) => prob: 0.39158 / x: 266 / y: 633
[pointed] LAnkle (13) => prob: 0.40900 / x: 292 / y: 798
[pointed] REye (14) => prob: 0.36307 / x: 399 / y: 117
[not pointed] LEye (15) => prob: 0.01053 / x: 385 / y: 93
[pointed] REar (16) =

============================== frame: 15 / 153 ==============================
[not pointed] Nose (0) => prob: 0.02635 / x: 385 / y: 70
[pointed] Neck (1) => prob: 0.63981 / x: 292 / y: 187
[pointed] RShoulder (2) => prob: 0.53009 / x: 385 / y: 211
[pointed] RElbow (3) => prob: 0.70200 / x: 425 / y: 352
[pointed] RWrist (4) => prob: 0.27796 / x: 372 / y: 352
[pointed] LShoulder (5) => prob: 0.56330 / x: 199 / y: 187
[pointed] LElbow (6) => prob: 0.73616 / x: 159 / y: 305
[not pointed] LWrist (7) => prob: 0.09767 / x: 212 / y: 328
[pointed] RHip (8) => prob: 0.30630 / x: 332 / y: 493
[pointed] RKnee (9) => prob: 0.31747 / x: 319 / y: 610
[pointed] RAnkle (10) => prob: 0.62282 / x: 319 / y: 821
[pointed] LHip (11) => prob: 0.36676 / x: 212 / y: 469
[pointed] LKnee (12) => prob: 0.46675 / x: 252 / y: 610
[pointed] LAnkle (13) => prob: 0.46472 / x: 279 / y: 798
[not pointed] REye (14) => prob: 0.07224 / x: 385 / y: 70
[not pointed] LEye (15) => prob: 0.01606 / x: 385 / y: 70
[pointed] REar 

============================== frame: 20 / 153 ==============================
[not pointed] Nose (0) => prob: 0.01456 / x: 372 / y: 70
[pointed] Neck (1) => prob: 0.68908 / x: 292 / y: 187
[pointed] RShoulder (2) => prob: 0.52823 / x: 385 / y: 211
[pointed] RElbow (3) => prob: 0.70242 / x: 425 / y: 352
[pointed] RWrist (4) => prob: 0.25580 / x: 372 / y: 352
[pointed] LShoulder (5) => prob: 0.64338 / x: 199 / y: 187
[pointed] LElbow (6) => prob: 0.63381 / x: 159 / y: 328
[not pointed] LWrist (7) => prob: 0.09540 / x: 212 / y: 328
[pointed] RHip (8) => prob: 0.39365 / x: 332 / y: 493
[pointed] RKnee (9) => prob: 0.33569 / x: 319 / y: 633
[pointed] RAnkle (10) => prob: 0.55312 / x: 319 / y: 821
[pointed] LHip (11) => prob: 0.34519 / x: 212 / y: 469
[pointed] LKnee (12) => prob: 0.43565 / x: 252 / y: 610
[pointed] LAnkle (13) => prob: 0.43523 / x: 279 / y: 821
[not pointed] REye (14) => prob: 0.04543 / x: 372 / y: 70
[not pointed] LEye (15) => prob: 0.00828 / x: 372 / y: 70
[pointed] REar 

...

============================== frame: 130 / 153 ==============================
[pointed] Nose (0) => prob: 0.15404 / x: 425 / y: 187
[pointed] Neck (1) => prob: 0.64678 / x: 385 / y: 164
[pointed] RShoulder (2) => prob: 0.75731 / x: 399 / y: 187
[pointed] RElbow (3) => prob: 0.57005 / x: 372 / y: 211
[pointed] RWrist (4) => prob: 0.61015 / x: 359 / y: 258
[pointed] LShoulder (5) => prob: 0.50518 / x: 385 / y: 164
[pointed] LElbow (6) => prob: 0.20927 / x: 332 / y: 164
[not pointed] LWrist (7) => prob: 0.03679 / x: 412 / y: 211
[pointed] RHip (8) => prob: 0.67216 / x: 306 / y: 234
[pointed] RKnee (9) => prob: 0.79102 / x: 332 / y: 328
[pointed] RAnkle (10) => prob: 0.81691 / x: 292 / y: 422
[pointed] LHip (11) => prob: 0.45437 / x: 279 / y: 211
[pointed] LKnee (12) => prob: 0.18215 / x: 319 / y: 305
[pointed] LAnkle (13) => prob: 0.11827 / x: 292 / y: 399
[pointed] REye (14) => prob: 0.19610 / x: 425 / y: 164
[not pointed] LEye (15) => prob: 0.01964 / x: 439 / y: 187
[pointed] REar (16)

============================== frame: 135 / 153 ==============================
[not pointed] Nose (0) => prob: 0.03992 / x: 439 / y: 164
[pointed] Neck (1) => prob: 0.85755 / x: 399 / y: 164
[pointed] RShoulder (2) => prob: 0.74756 / x: 412 / y: 187
[pointed] RElbow (3) => prob: 0.69079 / x: 385 / y: 234
[pointed] RWrist (4) => prob: 0.45430 / x: 372 / y: 258
[pointed] LShoulder (5) => prob: 0.67122 / x: 372 / y: 140
[pointed] LElbow (6) => prob: 0.69790 / x: 332 / y: 117
[pointed] LWrist (7) => prob: 0.10350 / x: 332 / y: 117
[pointed] RHip (8) => prob: 0.56447 / x: 332 / y: 234
[pointed] RKnee (9) => prob: 0.56982 / x: 372 / y: 328
[pointed] RAnkle (10) => prob: 0.29654 / x: 372 / y: 328
[pointed] LHip (11) => prob: 0.61214 / x: 292 / y: 211
[pointed] LKnee (12) => prob: 0.53323 / x: 319 / y: 328
[pointed] LAnkle (13) => prob: 0.64377 / x: 279 / y: 375
[not pointed] REye (14) => prob: 0.07669 / x: 439 / y: 164
[not pointed] LEye (15) => prob: 0.01890 / x: 439 / y: 140
[pointed] REar 

============================== frame: 140 / 153 ==============================
[not pointed] Nose (0) => prob: 0.02930 / x: 412 / y: 93
[pointed] Neck (1) => prob: 0.59776 / x: 412 / y: 140
[pointed] RShoulder (2) => prob: 0.54008 / x: 425 / y: 164
[pointed] RElbow (3) => prob: 0.18620 / x: 412 / y: 211
[not pointed] RWrist (4) => prob: 0.07899 / x: 385 / y: 305
[pointed] LShoulder (5) => prob: 0.49049 / x: 385 / y: 117
[pointed] LElbow (6) => prob: 0.24348 / x: 345 / y: 140
[pointed] LWrist (7) => prob: 0.10396 / x: 332 / y: 164
[pointed] RHip (8) => prob: 0.39249 / x: 359 / y: 234
[pointed] RKnee (9) => prob: 0.63822 / x: 306 / y: 305
[pointed] RAnkle (10) => prob: 0.66883 / x: 239 / y: 328
[pointed] LHip (11) => prob: 0.42630 / x: 345 / y: 211
[pointed] LKnee (12) => prob: 0.30713 / x: 306 / y: 305
[pointed] LAnkle (13) => prob: 0.15190 / x: 239 / y: 328
[not pointed] REye (14) => prob: 0.02762 / x: 412 / y: 234
[not pointed] LEye (15) => prob: 0.03148 / x: 412 / y: 93
[pointed] REa

============================== frame: 145 / 153 ==============================
[not pointed] Nose (0) => prob: 0.04741 / x: 412 / y: 93
[pointed] Neck (1) => prob: 0.12576 / x: 412 / y: 117
[not pointed] RShoulder (2) => prob: 0.07901 / x: 385 / y: 117
[not pointed] RElbow (3) => prob: 0.05215 / x: 332 / y: 164
[not pointed] RWrist (4) => prob: 0.04369 / x: 345 / y: 164
[not pointed] LShoulder (5) => prob: 0.07683 / x: 399 / y: 117
[not pointed] LElbow (6) => prob: 0.05279 / x: 332 / y: 164
[not pointed] LWrist (7) => prob: 0.02991 / x: 399 / y: 211
[pointed] RHip (8) => prob: 0.22266 / x: 359 / y: 211
[pointed] RKnee (9) => prob: 0.49380 / x: 332 / y: 281
[pointed] RAnkle (10) => prob: 0.52491 / x: 252 / y: 281
[pointed] LHip (11) => prob: 0.20488 / x: 359 / y: 211
[pointed] LKnee (12) => prob: 0.26286 / x: 332 / y: 281
[pointed] LAnkle (13) => prob: 0.24704 / x: 239 / y: 281
[not pointed] REye (14) => prob: 0.03758 / x: 425 / y: 93
[not pointed] LEye (15) => prob: 0.04914 / x: 412 / 

============================== frame: 150 / 153 ==============================
[not pointed] Nose (0) => prob: 0.05969 / x: 399 / y: 46
[pointed] Neck (1) => prob: 0.10793 / x: 399 / y: 93
[pointed] RShoulder (2) => prob: 0.11581 / x: 372 / y: 93
[pointed] RElbow (3) => prob: 0.10242 / x: 332 / y: 164
[not pointed] RWrist (4) => prob: 0.07916 / x: 345 / y: 164
[pointed] LShoulder (5) => prob: 0.10032 / x: 332 / y: 164
[not pointed] LElbow (6) => prob: 0.06573 / x: 345 / y: 164
[not pointed] LWrist (7) => prob: 0.05445 / x: 345 / y: 140
[pointed] RHip (8) => prob: 0.13747 / x: 359 / y: 211
[pointed] RKnee (9) => prob: 0.31029 / x: 359 / y: 234
[pointed] RAnkle (10) => prob: 0.50787 / x: 372 / y: 305
[pointed] LHip (11) => prob: 0.11155 / x: 359 / y: 187
[pointed] LKnee (12) => prob: 0.15623 / x: 359 / y: 234
[pointed] LAnkle (13) => prob: 0.15744 / x: 372 / y: 281
[not pointed] REye (14) => prob: 0.05295 / x: 385 / y: 46
[not pointed] LEye (15) => prob: 0.04999 / x: 399 / y: 46
[not poi

In [47]:
user = "C:/Users/yuddo/Yuchul/Project/Bowling/data/user/1.mp4"

output_keypoints_with_lines_video(proto_file=protoFile_coco, weights_file=weightsFile_coco, video_path=user,
                                  threshold=0.1, BODY_PARTS=BODY_PARTS_COCO, POSE_PAIRS=POSE_PAIRS_COCO)

============================== frame: 0 / 213 ==============================
[not pointed] Nose (0) => prob: 0.00776 / x: 516 / y: 292
[pointed] Neck (1) => prob: 0.61631 / x: 328 / y: 459
[pointed] RShoulder (2) => prob: 0.57818 / x: 516 / y: 500
[pointed] RElbow (3) => prob: 0.52000 / x: 516 / y: 876
[pointed] RWrist (4) => prob: 0.40946 / x: 540 / y: 1001
[pointed] LShoulder (5) => prob: 0.59455 / x: 140 / y: 459
[pointed] LElbow (6) => prob: 0.13105 / x: 70 / y: 793
[not pointed] LWrist (7) => prob: 0.01889 / x: 540 / y: 918
[pointed] RHip (8) => prob: 0.34398 / x: 375 / y: 1085
[pointed] RKnee (9) => prob: 0.43507 / x: 375 / y: 1460
[pointed] RAnkle (10) => prob: 0.37616 / x: 328 / y: 1836
[pointed] LHip (11) => prob: 0.35628 / x: 187 / y: 1085
[pointed] LKnee (12) => prob: 0.63775 / x: 234 / y: 1419
[pointed] LAnkle (13) => prob: 0.37624 / x: 281 / y: 1836
[not pointed] REye (14) => prob: 0.02371 / x: 493 / y: 208
[not pointed] LEye (15) => prob: 0.01661 / x: 305 / y: 250
[pointe

============================== frame: 5 / 213 ==============================
[not pointed] Nose (0) => prob: 0.00873 / x: 516 / y: 292
[pointed] Neck (1) => prob: 0.60072 / x: 352 / y: 459
[pointed] RShoulder (2) => prob: 0.59947 / x: 540 / y: 500
[pointed] RElbow (3) => prob: 0.56174 / x: 516 / y: 876
[pointed] RWrist (4) => prob: 0.52512 / x: 540 / y: 1001
[pointed] LShoulder (5) => prob: 0.58866 / x: 140 / y: 459
[pointed] LElbow (6) => prob: 0.10879 / x: 93 / y: 793
[not pointed] LWrist (7) => prob: 0.01675 / x: 563 / y: 918
[pointed] RHip (8) => prob: 0.36539 / x: 375 / y: 1085
[pointed] RKnee (9) => prob: 0.44995 / x: 375 / y: 1460
[pointed] RAnkle (10) => prob: 0.38081 / x: 328 / y: 1878
[pointed] LHip (11) => prob: 0.35689 / x: 187 / y: 1043
[pointed] LKnee (12) => prob: 0.62180 / x: 234 / y: 1419
[pointed] LAnkle (13) => prob: 0.36945 / x: 281 / y: 1836
[not pointed] REye (14) => prob: 0.02545 / x: 516 / y: 250
[not pointed] LEye (15) => prob: 0.02201 / x: 305 / y: 250
[pointe

============================== frame: 10 / 213 ==============================
[not pointed] Nose (0) => prob: 0.01490 / x: 516 / y: 250
[pointed] Neck (1) => prob: 0.58822 / x: 328 / y: 459
[pointed] RShoulder (2) => prob: 0.60284 / x: 540 / y: 500
[pointed] RElbow (3) => prob: 0.55958 / x: 516 / y: 834
[pointed] RWrist (4) => prob: 0.56927 / x: 540 / y: 1001
[pointed] LShoulder (5) => prob: 0.58932 / x: 140 / y: 459
[pointed] LElbow (6) => prob: 0.18356 / x: 70 / y: 793
[not pointed] LWrist (7) => prob: 0.01697 / x: 117 / y: 1001
[pointed] RHip (8) => prob: 0.37234 / x: 375 / y: 1085
[pointed] RKnee (9) => prob: 0.47030 / x: 375 / y: 1460
[pointed] RAnkle (10) => prob: 0.37444 / x: 328 / y: 1878
[pointed] LHip (11) => prob: 0.35994 / x: 187 / y: 1043
[pointed] LKnee (12) => prob: 0.62012 / x: 234 / y: 1419
[pointed] LAnkle (13) => prob: 0.36064 / x: 281 / y: 1836
[not pointed] REye (14) => prob: 0.04696 / x: 516 / y: 250
[not pointed] LEye (15) => prob: 0.01513 / x: 305 / y: 250
[poin

============================== frame: 15 / 213 ==============================
[not pointed] Nose (0) => prob: 0.01967 / x: 516 / y: 292
[pointed] Neck (1) => prob: 0.62492 / x: 352 / y: 459
[pointed] RShoulder (2) => prob: 0.60849 / x: 540 / y: 500
[pointed] RElbow (3) => prob: 0.64837 / x: 516 / y: 834
[pointed] RWrist (4) => prob: 0.58721 / x: 540 / y: 1001
[pointed] LShoulder (5) => prob: 0.58487 / x: 140 / y: 459
[pointed] LElbow (6) => prob: 0.15794 / x: 93 / y: 793
[not pointed] LWrist (7) => prob: 0.01888 / x: 117 / y: 1001
[pointed] RHip (8) => prob: 0.38869 / x: 375 / y: 1085
[pointed] RKnee (9) => prob: 0.43127 / x: 375 / y: 1460
[pointed] RAnkle (10) => prob: 0.37236 / x: 328 / y: 1836
[pointed] LHip (11) => prob: 0.37677 / x: 187 / y: 1043
[pointed] LKnee (12) => prob: 0.69315 / x: 234 / y: 1419
[pointed] LAnkle (13) => prob: 0.28805 / x: 281 / y: 1836
[not pointed] REye (14) => prob: 0.07043 / x: 516 / y: 250
[not pointed] LEye (15) => prob: 0.01741 / x: 328 / y: 250
[poin

============================== frame: 20 / 213 ==============================
[not pointed] Nose (0) => prob: 0.01455 / x: 516 / y: 292
[pointed] Neck (1) => prob: 0.65454 / x: 352 / y: 459
[pointed] RShoulder (2) => prob: 0.60569 / x: 540 / y: 500
[pointed] RElbow (3) => prob: 0.64547 / x: 516 / y: 834
[pointed] RWrist (4) => prob: 0.55039 / x: 516 / y: 1001
[pointed] LShoulder (5) => prob: 0.54706 / x: 140 / y: 459
[pointed] LElbow (6) => prob: 0.15890 / x: 117 / y: 793
[not pointed] LWrist (7) => prob: 0.01721 / x: 117 / y: 1001
[pointed] RHip (8) => prob: 0.38505 / x: 375 / y: 1085
[pointed] RKnee (9) => prob: 0.39377 / x: 375 / y: 1460
[pointed] RAnkle (10) => prob: 0.37513 / x: 328 / y: 1836
[pointed] LHip (11) => prob: 0.37738 / x: 187 / y: 1043
[pointed] LKnee (12) => prob: 0.67665 / x: 234 / y: 1419
[pointed] LAnkle (13) => prob: 0.24046 / x: 281 / y: 1711
[not pointed] REye (14) => prob: 0.04898 / x: 516 / y: 250
[not pointed] LEye (15) => prob: 0.00676 / x: 516 / y: 292
[poi

...

============================== frame: 190 / 213 ==============================
[pointed] Nose (0) => prob: 0.67757 / x: 540 / y: 834
[pointed] Neck (1) => prob: 0.75062 / x: 493 / y: 876
[pointed] RShoulder (2) => prob: 0.42813 / x: 469 / y: 834
[pointed] RElbow (3) => prob: 0.44536 / x: 399 / y: 834
[pointed] RWrist (4) => prob: 0.41326 / x: 305 / y: 876
[pointed] LShoulder (5) => prob: 0.45494 / x: 493 / y: 876
[pointed] LElbow (6) => prob: 0.31657 / x: 399 / y: 834
[pointed] LWrist (7) => prob: 0.17791 / x: 305 / y: 876
[pointed] RHip (8) => prob: 0.68775 / x: 399 / y: 918
[pointed] RKnee (9) => prob: 0.84437 / x: 352 / y: 1043
[pointed] RAnkle (10) => prob: 0.77027 / x: 258 / y: 1085
[pointed] LHip (11) => prob: 0.75183 / x: 422 / y: 960
[pointed] LKnee (12) => prob: 0.56098 / x: 446 / y: 1085
[pointed] LAnkle (13) => prob: 0.39111 / x: 446 / y: 1085
[pointed] REye (14) => prob: 0.70886 / x: 540 / y: 834
[not pointed] LEye (15) => prob: 0.04694 / x: 540 / y: 834
[pointed] REar (16)

============================== frame: 195 / 213 ==============================
[pointed] Nose (0) => prob: 0.59843 / x: 540 / y: 834
[pointed] Neck (1) => prob: 0.73250 / x: 493 / y: 876
[pointed] RShoulder (2) => prob: 0.61790 / x: 469 / y: 834
[pointed] RElbow (3) => prob: 0.53339 / x: 399 / y: 834
[pointed] RWrist (4) => prob: 0.40951 / x: 399 / y: 876
[pointed] LShoulder (5) => prob: 0.55090 / x: 493 / y: 876
[pointed] LElbow (6) => prob: 0.19275 / x: 540 / y: 876
[pointed] LWrist (7) => prob: 0.14968 / x: 540 / y: 834
[pointed] RHip (8) => prob: 0.75641 / x: 399 / y: 960
[pointed] RKnee (9) => prob: 0.82895 / x: 352 / y: 1043
[pointed] RAnkle (10) => prob: 0.71540 / x: 258 / y: 1085
[pointed] LHip (11) => prob: 0.73911 / x: 422 / y: 960
[pointed] LKnee (12) => prob: 0.52595 / x: 446 / y: 1085
[pointed] LAnkle (13) => prob: 0.43303 / x: 446 / y: 1085
[pointed] REye (14) => prob: 0.50732 / x: 540 / y: 793
[pointed] LEye (15) => prob: 0.16300 / x: 540 / y: 834
[pointed] REar (16) => 

============================== frame: 200 / 213 ==============================
[pointed] Nose (0) => prob: 0.32649 / x: 516 / y: 876
[pointed] Neck (1) => prob: 0.78521 / x: 493 / y: 876
[pointed] RShoulder (2) => prob: 0.71050 / x: 446 / y: 834
[pointed] RElbow (3) => prob: 0.68389 / x: 399 / y: 834
[pointed] RWrist (4) => prob: 0.64850 / x: 328 / y: 876
[pointed] LShoulder (5) => prob: 0.67614 / x: 516 / y: 876
[pointed] LElbow (6) => prob: 0.51954 / x: 563 / y: 876
[pointed] LWrist (7) => prob: 0.65213 / x: 540 / y: 834
[pointed] RHip (8) => prob: 0.71425 / x: 399 / y: 960
[pointed] RKnee (9) => prob: 0.84985 / x: 352 / y: 1043
[pointed] RAnkle (10) => prob: 0.73473 / x: 281 / y: 1126
[pointed] LHip (11) => prob: 0.75487 / x: 422 / y: 1001
[pointed] LKnee (12) => prob: 0.53990 / x: 446 / y: 1085
[pointed] LAnkle (13) => prob: 0.32465 / x: 422 / y: 1126
[pointed] REye (14) => prob: 0.30049 / x: 516 / y: 834
[pointed] LEye (15) => prob: 0.22903 / x: 516 / y: 834
[pointed] REar (16) =>

============================== frame: 205 / 213 ==============================
[pointed] Nose (0) => prob: 0.45676 / x: 540 / y: 834
[pointed] Neck (1) => prob: 0.74466 / x: 493 / y: 876
[pointed] RShoulder (2) => prob: 0.72946 / x: 469 / y: 834
[pointed] RElbow (3) => prob: 0.61700 / x: 399 / y: 876
[pointed] RWrist (4) => prob: 0.64062 / x: 328 / y: 876
[pointed] LShoulder (5) => prob: 0.60540 / x: 516 / y: 876
[pointed] LElbow (6) => prob: 0.34691 / x: 563 / y: 918
[pointed] LWrist (7) => prob: 0.51973 / x: 563 / y: 876
[pointed] RHip (8) => prob: 0.72989 / x: 399 / y: 960
[pointed] RKnee (9) => prob: 0.78772 / x: 352 / y: 1043
[pointed] RAnkle (10) => prob: 0.77887 / x: 281 / y: 1126
[pointed] LHip (11) => prob: 0.76546 / x: 422 / y: 1001
[pointed] LKnee (12) => prob: 0.45779 / x: 446 / y: 1085
[pointed] LAnkle (13) => prob: 0.45740 / x: 422 / y: 1126
[pointed] REye (14) => prob: 0.46324 / x: 540 / y: 834
[pointed] LEye (15) => prob: 0.27808 / x: 540 / y: 834
[pointed] REar (16) =>

============================== frame: 210 / 213 ==============================
[pointed] Nose (0) => prob: 0.33661 / x: 540 / y: 876
[pointed] Neck (1) => prob: 0.68933 / x: 493 / y: 876
[pointed] RShoulder (2) => prob: 0.68429 / x: 493 / y: 876
[pointed] RElbow (3) => prob: 0.48938 / x: 540 / y: 918
[pointed] RWrist (4) => prob: 0.45500 / x: 563 / y: 876
[pointed] LShoulder (5) => prob: 0.47186 / x: 469 / y: 876
[pointed] LElbow (6) => prob: 0.22012 / x: 399 / y: 834
[pointed] LWrist (7) => prob: 0.10620 / x: 563 / y: 918
[pointed] RHip (8) => prob: 0.70950 / x: 399 / y: 960
[pointed] RKnee (9) => prob: 0.75201 / x: 352 / y: 1085
[pointed] RAnkle (10) => prob: 0.81478 / x: 281 / y: 1126
[pointed] LHip (11) => prob: 0.72610 / x: 422 / y: 1001
[pointed] LKnee (12) => prob: 0.59137 / x: 422 / y: 1085
[pointed] LAnkle (13) => prob: 0.55548 / x: 422 / y: 1126
[pointed] REye (14) => prob: 0.30952 / x: 540 / y: 876
[not pointed] LEye (15) => prob: 0.07383 / x: 540 / y: 876
[pointed] REar (16

In [50]:
def split_video_into_frames(video_path, output_folder):
    # 비디오 읽어오기
    capture = cv2.VideoCapture(video_path)

    frame_count = 0

    while True:
        ret, frame = capture.read()
        if not ret:
            break

        frame_path = f"{output_folder}/frame_{frame_count}.jpg"
        cv2.imwrite(frame_path, frame)

        frame_count += 1

    capture.release()

In [71]:
# 샴 네트워크 모델 정의
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(1, 4, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(4),

            nn.ReflectionPad2d(1),
            nn.Conv2d(4, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),

            nn.ReflectionPad2d(1),
            nn.Conv2d(8, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),
        )

        self.fc1 = nn.Sequential(
            nn.Linear(8*100*100, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 5))

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2
        
#     def forward(self, input1, input2):
#         # 입력 이미지에 대한 처리 수행
#         # 각 이미지를 네트워크에 통과시킴
#         output1 = self.model(input1)
#         output2 = self.model(input2)
        
#         # 이미지 쌍의 유사성 계산
#         # 예를 들어, 유클리드 거리 계산 등
        
#         return similarity_score

# 동영상 데이터 처리
user_video_path = "C:/Users/yuddo/Yuchul/Project/Bowling/data/output/user.avi"
pro_video_path = "C:/Users/yuddo/Yuchul/Project/Bowling/data/output/pro.avi"
# 저장 폴더 경로 정의
user_output_folder = "C:/Users/yuddo/Yuchul/Project/Bowling/data/output/user_frame_img"
pro_output_folder = "C:/Users/yuddo/Yuchul/Project/Bowling/data/output/pro_frame_img"

In [69]:
class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
        (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [57]:
# 동영상을 프레임 단위로 분할
user_video_frames = split_video_into_frames(user_video_path, user_output_folder)

In [60]:
pro_video_frames = split_video_into_frames(pro_video_path, pro_output_folder)

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 폴더 경로 정의
folder = "C:/Users/yuddo/Yuchul/Project/Bowling/data/output"

# 이미지 폴더 데이터셋 정의
user_dataset = ImageFolder(folder, transform=ToTensor())
pro_dataset = ImageFolder(folder, transform=ToTensor())

# 데이터 로더 정의
user_loader = DataLoader(user_dataset, batch_size=2, shuffle=False)
pro_loader = DataLoader(pro_dataset, batch_size=1, shuffle=False)

# 모델 정의 및 불러오기
model = SiameseNetwork().to(device)

# 샴 네트워크에 이미지 프레임 적용
similarity_scores = []
for (image1, _), (image2, _) in zip(user_loader, pro_loader):
    image1 = image1.to(device)
    image2 = image2.to(device)
    
    output = model(image1, image2)
    similarity_scores.append(output.item())

# 결과 출력
for i, similarity_score in enumerate(similarity_scores):
    print(f"Frame {i+1}: Similarity Score = {similarity_score}")


AttributeError: 'SiameseNetwork' object has no attribute 'model'